In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.graph_objs as go
import glob
import os

In [96]:
DUMP_ID = 20

In [97]:
PATH_PREFIX = "/mnt/storage/data/vision/imagenet/training_dumps/" + str(DUMP_ID).zfill(8)

### Parameters

In [11]:
def get_values_from_files(filenames):
    arr = []
    for f in filenames:
        arr.append(np.fromfile(f, dtype=np.float32))
    return arr

In [98]:
model_param_buffer_filenames = sorted(glob.glob(PATH_PREFIX + "/model_params/*"))
gradient_buffer_filenames = sorted(glob.glob(PATH_PREFIX + "/gradients/*"))
mean_buffer_filenames = sorted(glob.glob(PATH_PREFIX + "/means/*"))
var_buffer_filenames = sorted(glob.glob(PATH_PREFIX + "/vars/*"))

In [99]:
model_params_list = get_values_from_files(model_param_buffer_filenames)
gradients_list = get_values_from_files(gradient_buffer_filenames)
means_list = get_values_from_files(mean_buffer_filenames)
vars_list = get_values_from_files(var_buffer_filenames)

In [100]:
for i in range(212, -1, -1):
    print("[" + str(min(gradients_list[i])) + "-" + str(max(gradients_list[i])) + "]")

[-7.346592-0.24462497]
[-0.1179582-0.14559618]
[-0.0721461-0.08362873]
[0.0-0.0]
[-1.1489965-1.0923663]
[-0.48844644-0.4247844]
[-0.27774802-0.29459515]
[0.0-0.0]
[-1.2510228-1.0953969]
[-0.6572272-0.523505]
[-0.42086893-0.46815622]
[0.0-0.0]
[-7.1746826-7.5405064]
[-0.5009949-0.39637223]
[-0.21484597-0.2040641]
[0.0-0.0]
[-3.2134016-3.5778515]
[-1.0729384-1.2865623]
[-0.8267661-0.9580863]
[0.0-0.0]
[-3.4770713-3.3286843]
[-1.6324428-1.7588785]
[-1.2719656-1.1485192]
[0.0-0.0]
[-15.950534-16.685757]
[-1.2159746-1.2504745]
[-0.62604916-0.69551206]
[0.0-0.0]
[-5.6940103-5.5520563]
[-1.2159746-1.2504745]
[-0.5489419-0.5905269]
[0.0-0.0]
[-9.995241-8.875591]
[-3.309212-3.8074222]
[-2.4537017-2.6209211]
[0.0-0.0]
[-21.98453-17.283842]
[-6.0278697-5.2382464]
[-4.683412-4.2432613]
[0.0-0.0]
[-148.7459-174.04509]
[-5.2730937-4.0930133]
[-1.46336-1.1081657]
[0.0-0.0]
[-61.31696-67.43459]
[-12.254918-13.188122]
[-8.962111-8.164194]
[0.0-0.0]
[-61.75834-65.56996]
[-19.721317-14.097773]
[-12.92719

### Forward Activations

In [18]:
def gather_batch_norm_data(dir_name):
    data_list = []
    for s in ["reduced", "spatial", "expanded"]:
        for f in ["means.buffer", "vars.buffer", "norm_temp.buffer", "norm.buffer"]:
            data = np.fromfile(dir_name + "/" + s + "/" + f, dtype=np.float32)
            data_list.append(data)
    return data_list

In [19]:
def gather_convblock_data(dir_name):
    data_list = []
    for f in ["reduction_applied.buffer", "reduction_activated.buffer", "spatial_applied.buffer", "spatial_activated.buffer", "expanded_applied.buffer", "combined_output.buffer", "output_activated.buffer"]:
        data = np.fromfile(dir_name + "/" + f, dtype=np.float32)
        data_list.append(data)
    return data_list

#### Initial Conv Layer and MaxPool

In [66]:
init_conv_applied = np.fromfile(PATH_PREFIX + "/activations/init_conv_applied.buffer", dtype=np.float32)
init_conv_activated = np.fromfile(PATH_PREFIX + "/activations/init_conv_activated.buffer", dtype=np.float32)
max_inds = np.fromfile(PATH_PREFIX + "/activations/max_inds.buffer", dtype=np.int32)
init_convblock_input = np.fromfile(PATH_PREFIX + "/activations/init_convblock_input.buffer", dtype=np.float32)

In [68]:
data_list = gather_batch_norm_data(PATH_PREFIX + "/activations/batch_norms/00")

In [69]:
conv_data_list = gather_convblock_data(PATH_PREFIX + "/activations/conv_blocks/15")

#### Final Avg Pool, Fully Connected, and Softmax

In [67]:
final_avg_pool = np.fromfile(PATH_PREFIX + "/activations/final_avg_pool.buffer", dtype=np.float32)
fc_output = np.fromfile(PATH_PREFIX + "/activations/fc_output.buffer", dtype=np.float32)
prediction = np.fromfile(PATH_PREFIX + "/activations/softmax.buffer", dtype=np.float32)
correct_classes = np.fromfile(PATH_PREFIX + "/activations/correct_classes.buffer", dtype=np.int32)

In [57]:
def do_final_avg_pool(values):
    out = np.empty((32, 2048))
    for s in range(32):
        for c in range(2048):
            total = 0
            for i in range(7):
                for j in range(7):
                    total += values[s, i, j, c]
            avg_val = total / (7 * 7)
            out[s, c] = avg_val
    return out

In [70]:
final_conv_output_activated = np.reshape(conv_data_list[-1], (32, 7, 7, 2048))

In [71]:
cpu_final_avg_pool = do_final_avg_pool(final_conv_output_activated)

In [72]:
gpu_final_avg_pool = np.reshape(final_avg_pool, (32, 2048))

In [73]:
cpu_final_avg_pool

array([[0.32262802, 0.40709758, 0.32425573, ..., 0.22280605, 0.31874868,
        0.10614133],
       [0.1417158 , 0.47451776, 0.22778375, ..., 0.526419  , 0.55559315,
        0.43875361],
       [0.51229633, 0.2663506 , 0.18092323, ..., 0.42836551, 0.35719047,
        0.25173681],
       ...,
       [0.63864847, 0.08726742, 1.01121672, ..., 0.45842764, 0.49730837,
        0.14051307],
       [0.32922438, 0.26500626, 0.42306894, ..., 0.18250383, 0.53327359,
        0.21735177],
       [0.4974528 , 0.26396107, 0.40552521, ..., 0.23074147, 0.47229889,
        0.3946624 ]])

In [74]:
gpu_final_avg_pool

array([[0.32262802, 0.40709752, 0.32425573, ..., 0.22280604, 0.31874868,
        0.10614134],
       [0.1417158 , 0.47451773, 0.22778372, ..., 0.526419  , 0.5555932 ,
        0.43875363],
       [0.5122964 , 0.2663506 , 0.18092322, ..., 0.4283655 , 0.3571905 ,
        0.25173685],
       ...,
       [0.6386485 , 0.08726743, 1.0112166 , ..., 0.45842758, 0.49730843,
        0.14051308],
       [0.32922438, 0.26500627, 0.4230689 , ..., 0.18250383, 0.5332735 ,
        0.21735176],
       [0.4974528 , 0.26396105, 0.40552518, ..., 0.2307415 , 0.47229886,
        0.39466235]], dtype=float32)

### Backward Activations

In [436]:
d_init_conv_applied = np.fromfile(PATH_PREFIX + "/activation_derivs/init_conv_applied.buffer", dtype=np.float32)
d_init_conv_activated = np.fromfile(PATH_PREFIX + "/activation_derivs/init_conv_activated.buffer", dtype=np.float32)
d_init_convblock_input = np.fromfile(PATH_PREFIX + "/activation_derivs/init_convblock_input.buffer", dtype=np.float32)
d_final_avg_pool = np.fromfile(PATH_PREFIX + "/activation_derivs/final_avg_pool.buffer", dtype=np.float32)
d_fc_output = np.fromfile(PATH_PREFIX + "/activation_derivs/fc_output.buffer", dtype=np.float32)
d_prediction = np.fromfile(PATH_PREFIX + "/activation_derivs/softmax.buffer", dtype=np.float32)

In [438]:
print("[" + str(min(d_final_avg_pool)) + " - " + str(max(d_final_avg_pool)) + "]")

[-0.00401514 - 0.004302698]


In [486]:
DUMP_ID = 0
PATH_PREFIX = "/mnt/storage/data/vision/imagenet/training_dumps/" + str(DUMP_ID).zfill(8)
conv_data_list = gather_convblock_data(PATH_PREFIX + "/activation_derivs/conv_blocks/08")

In [487]:
for c in conv_data_list:
    print("[" + str(min(c)) + " - " + str(max(c)) + "]")

[-3.4619005 - 2.903917]
[-0.17878185 - 0.17705059]
[-0.75275075 - 0.84935224]
[-0.082793474 - 0.08686427]
[-0.6543597 - 0.62088454]
[-0.025375048 - 0.028530437]
[-0.025375048 - 0.028530437]


In [488]:
DUMP_ID = 2
PATH_PREFIX = "/mnt/storage/data/vision/imagenet/training_dumps/" + str(DUMP_ID).zfill(8)
conv_data_list = gather_convblock_data(PATH_PREFIX + "/activation_derivs/conv_blocks/08")

In [489]:
for c in conv_data_list:
    print("[" + str(min(c)) + " - " + str(max(c)) + "]")

[-3.0024638 - 3.4002678]
[-0.18356527 - 0.17030282]
[-0.8715495 - 0.8251272]
[-0.1011737 - 0.08904381]
[-0.74025804 - 0.6807653]
[-0.027166499 - 0.028757486]
[-0.027166499 - 0.028757486]
